In [ ]:
from PyQt5.QtWidgets import QWidget, QCheckBox, QApplication
from PyQt5.QtCore import Qt, pyqtSignal
from glob import glob
import sys

#### this works but the window title doesn't change when clicked 

In [ ]:
p = '/active_projects/anthony/ns650/40039102'
files = glob(p + '/*cnt')

files_len  = len(files)

app = QApplication(sys.argv)
widget = QWidget()

widget.setGeometry(400, 200, 800, 600)
widget.setWindowTitle('This is a window title')

a=0
cb = []
for i in files:
    cb.append(QCheckBox(i, widget))
    #a = a + 1

# this a counter moves each line in interation to the next line 
a = 0
for i in files:
    cb[a].move(10, 20*a)
    a=a+1
    
def hello():
    
    for i in range(0, files_len):
        if state == cb[i].isChecked():
            widget.setWindowTitle(files[i])
            #break;
        else:
            widget.setWindowTitle('Box was checked')
            
#for i in range(0, files_len):
    #QCheckBox.stateChanged.connect(cb[i], pyqtSignal('stateChanged(int)'), hello)
    
widget.show()
sys.exit(app.exec_())

#### write the program first then put into a class??

In [ ]:
class myclass(QWidget):
    
    def __init__(self):
        super().__init__()
        self.initUI()
        
    def get_files(self, path):
        self.path = path
        
        files = glob(path + '/*cnt')
        return files 
    
    def initUi(self, files):
        self.files = files
        
        app = QApplication(sys.argv)
        widget = QWidget()

        widget.setGeometry(400, 200, 800, 600)
        widget.setWindowTitle('H1 files')
        
        a=0
        cb = []
        for i in files:
            cb.append(QCheckBox(i, widget))
            a = a + 1
            
        a = 0
        for i in files:
            cb[a].move(10, 20*a)
            a=a+1
            
        for i in range(0, files_len):
            if cb[i].isChecked():
                widget.setWindowTitle(files[i])
                break;
            else:
                widget.setWindowTitle('None')

        
        
        
  

https://pythonspot.com/pyqt5-input-dialog/

#### this is the box prompt to get the path -- feed this into H1 maker class


##### why does another window pop up after pressing OK

In [1]:
from PyQt5.QtWidgets import QInputDialog, QLineEdit, QApplication, QWidget
import sys

class App(QWidget):
 
    def __init__(self):
        super().__init__()
        #self.title = 'H1 file maker'
        self.left = 10
        self.top = 10
        self.width = 640
        self.height = 480
        self.initUI()
 
    def initUI(self):
        #self.setWindowTitle(self.title)
        self.setGeometry(self.left, self.top, self.width, self.height)
        self.getText()
 
        self.show()
 
 
    def getText(self):
        
        dir_path = []
        text, okPressed = QInputDialog.getText(self, "HBNL Peak Picker","Enter path of files to be picked:", QLineEdit.Normal, "")
        if okPressed and text != '':
            dir_path.append(text)
            pass
        else:
            print("\nERROR: Box cannot be left blank before pressing OK ")
        print(dir_path)

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = App()
    sys.exit(app.exec_())

['path f files']


SystemExit: 0

/usr/local/anaconda3/envs/dbI/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
import os
import sys
from glob import glob
from collections import defaultdict
from collections import Counter
import collections
import re
import shutil
import subprocess
import pandas as pd

class site_data:
    #get_h1s()
    def __init__(self):
    
        self.ant_set = {'ant'}
        self.ans_set = {'ans'}
        self.other_exps = {'vp3', 'cpt', 'ern', 'aod', 'stp', 'gng'}
        
    #get_h1s()    
    def check_cnt_copy(self, fp_check_cnt):
        '''create dictionary of lists where id is key and exp*cnt names are the values'''
    
        self.fp_check_cnt = fp_check_cnt

        cnt_dict = {}
        for r,d,f in os.walk(fp_check_cnt):
            for n in f:
                if n.endswith('_32.cnt'):
                    split = n.split('_')
                    cnt_dict.setdefault(split[3], []).append(split[0])
                    
        for k,v in cnt_dict.items():
            if len(v) is not 3:
                print('Check {} for experiments {}'.format(k,v))
    #get_h1s()                
    def rename_reruns(self, fp_rerun, fp_rerun_trg=None):
        '''if peak-picking, copy rr file to target directory and rename it.
           if not peak-picking, only let me know there is a rr file'''

        self.fp_rerun = fp_rerun
        self.fp_rerun_trg = fp_rerun_trg
        
        #check for _rr.cnt regardless of run number 
        for r,d,f in os.walk(fp_rerun):
            for n in f:
                if n.endswith('_rr.cnt'):
                    print('Found re-runs...',  os.path.basename(fp_rerun), n)
                    answer=input('Copy to target directory?')
                    if answer in ['y', 'Y', 'yes', 'Yes']:
                        shutil.copy(os.path.join(r,n), os.path.join(fp_rerun_trg, n[:3])) 
                    else:
                        pass
                    
        #rename only if peak picking              
        if fp_rerun_trg:
            for r,d,f in os.walk(fp_rerun_trg):
                for n in f:
                    if n.endswith('_rr.cnt'):
                        new_name = os.rename(os.path.join(r, n), os.path.join(r,n.replace("_rr", "")))
                        print('\n', 'Renaming', os.path.join(r,n), '\n', 
                              'to','\n', os.path.join(r,n.replace("_rr", ""), '\n'))
    #get_h1s()                    
    def create_cnth1(self, fp_cnt):
        '''given a filepath -- create _cnt.h1 files from shell script'''
        
        self.fp_cnt = fp_cnt
        
        print('\n', '_________________________________________________CREATING CNT.H1S_________________________________________________')
        for r,d,f in os.walk(fp_cnt):
            for n in f:
                if n.startswith(self.cnth1_tups) and n.endswith('_32.cnt'):
                    path = os.path.join(r,n)
                    p = subprocess.Popen("create_cnthdf1_from_cntneuroX.sh {}".format(path),shell=True, 
                                          stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd=os.path.dirname(path))
                    result = p.communicate()[1]
                    print(result.decode('ascii'))

    #get_h1s()
    def create_avgh1(self, fp_h1):
        '''given a filepath -- create _avg.h1 files from shell script'''
        
        self.fp_h1 = fp_h1

        print('\n', '_________________________________________________CREATING AVG.H1_________________________________________________')            
        for r,d,f in os.walk(fp_h1):
            for n in f:
                if n.startswith(self.ant_tup) and n.endswith('cnt.h1'):
                    ant_path = os.path.join(r,n)
                    p_ant = subprocess.Popen('create_avghdf1_from_cnthdf1X -lpfilter 8 -hpfilter 0.03 -thresh 75 -baseline_times -125 0 {}'.format(ant_path), 
                                             shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd=os.path.dirname(ant_path))
                    result_ant = p_ant.communicate()[1]
                    print(result_ant.decode('ascii'))
                if n.startswith(self.ans_tup) and n.endswith('cnt.h1'):
                    ans_path = os.path.join(r,n)
                    p_ans = subprocess.Popen('create_avghdf1_from_cnthdf1X -lpfilter 16 -hpfilter 0.03 -thresh 100 -baseline_times -125 0 {}'.format(ans_path), 
                                            shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd=os.path.dirname(ans_path))
                    result_ans = p_ans.communicate()[1]
                    print(result_ans.decode('ascii'))
                if n.startswith((self.others_tup)) and n.endswith('cnt.h1'):
                    path=os.path.join(r,n)
                    p_others = subprocess.Popen('create_avghdf1_from_cnthdf1X -lpfilter 16 -hpfilter 0.03 -thresh 75 -baseline_times -125 0 {}'.format(path), 
                                                shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd=os.path.dirname(path))
                    result_others = p_others.communicate()[1]
                    print(result_others.decode('ascii'))
    #get_h1s()
    def create_avgps(self, fp_ps):
        '''given a filepath -- create avg.h1.ps files from shell script'''
        
        self.fp_ps = fp_ps

        print('\n', '_________________________________________________CREATE H1.PS_________________________________________________')
        for path, subdirs, files in os.walk(fp_ps):
            for name in files:
                if name.endswith("avg.h1"):
                    ps_paths = os.path.join(path, name)
                    subprocess.Popen("plot_hdf1_data.sh {}".format(name), shell=True, cwd=os.path.dirname(ps_paths))
                    print("creating ps files.. " + name)
                    
    #get_h1s()                
    def delete_bad_files(self, fp, exts_to_keep=None, to_be_deleted_set=None):
        ''' returns any extension not in exts_to_keep & prompts user to delete '''
    
        self.fp = fp
        self.exts_to_keep = exts_to_keep
        self.to_be_deleted_set = to_be_deleted_set


        print('\n', '_________________________________________________CLEANING UP...________________________________________________')
        
        if exts_to_keep:

            count=0
            for r,d,f in os.walk(fp):
                for n in f:
                    if not n.endswith(exts_to_keep):
                        count+=1
                        path = os.path.join(r,n)
                        print(count,'||', path)
                        ans = input('^^Do you want to delete this file?\n')
                        if ans in ['y', 'Y', 'Yes', 'yes']:
                            os.remove(path)
                            print('Removing...', n, '\n\n')
                        else:
                            pass

        if to_be_deleted_set:

            del_tup = tuple(to_be_deleted_set)
            for r,d,f in os.walk(fp):
                for n in f:
                    if n.endswith(del_tup):
                        os.remove(os.path.join(r,n))
                        print('Removing...', n, '\n')

    #erp_peak_mover()
    def dir_exist(self, dir_path):
        '''checks if path exists and exits program if it does not'''
        
        self.dir_path = dir_path
        
        if os.path.isdir(dir_path):
            pass
        else:
            print('Path does not exist, check -->', dir_path)
            sys.exit(1)
            
    #erp_peak_mover()
    def concat_peak_paths(self, hbnl_dir, site):
        '''concats path to peak picked and reject directory for any site'''
        
        self.hbnl_dir = hbnl_dir
        self.site = site

        peak_pick_dirs = ['ant_phase4__peaks_2017', 'aod_phase4__peaks_2017', 'vp3_phase4__peaks_2017']
        hbnl = '/vol01/active_projects/HBNL/'

        good_dirs = [hbnl + '/' + i + '/' + site for i in peak_pick_dirs]
        rej_dirs = [hbnl + '/' + i + '/' + site + '/' + 'reject' for i in peak_pick_dirs]

        return good_dirs, rej_dirs
    
    def get_h1s(self, fp, set_of_exps, del_ext={}, ps=None, trg_dir=None):
        '''combines all these commands together'''
        
        self.fp = fp
        self.set_of_exps = set_of_exps
        self.del_ext = del_ext
        self.ps = ps
        self.trg_dir = trg_dir
        
        #use in create_cnth1()
        self.cnth1_tups = tuple(set_of_exps)
        
        if_ant = self.ant_set & set_of_exps
        if_ans = self.ans_set & set_of_exps
        all_others = self.other_exps & set_of_exps
        
        #used in create_avgh1()
        self.ant_tup = tuple(if_ant)
        self.ans_tup = tuple(if_ans)
        self.others_tup = tuple(all_others)

        #if being used for peak picking, create new directories and move all cnt files to the correct folder...and so on
        if trg_dir:
            #create new directories
            exps_list = list(set_of_exps)
            for exp in exps_list:
                new_dirs = os.path.join(trg_dir, exp)
                if not os.path.exists(new_dirs):
                    os.makedirs(new_dirs)
                    print("Creating " + new_dirs)
                else:
                    print(new_dirs + " already exist")
            #copy cnt files to newly created directories       
            count = 0
            for r,d,f in os.walk(fp):
                for n in f:
                    if n.startswith(self.cnth1_tups) and n.endswith('_32.cnt'):
                        count+=1
                        print('Copying ', n)
                        shutil.copy(os.path.join(r,n), os.path.join(trg_dir, n[:3]))
            print('Copied', count, 'files')
                        
            self.rename_reruns(fp,fp_rerun_trg=trg_dir)
            self.check_cnt_copy(trg_dir)
            self.create_cnth1(trg_dir)
            self.create_avgh1(trg_dir)
            if ps:
                self.create_avgps(trg_dir)
            if del_ext:
                self.delete_bad_files(trg_dir, to_be_deleted_set=del_ext)
            return True
        
        #if you want to create avg.h1s IN directory copying files...
        self.rename_reruns(fp)
        self.create_cnth1(fp)
        self.create_avgh1(fp)
        if ps:
            self.create_avgps(fp)
        if del_ext:
            self.delete_bad_files(fp, del_ext)

In [4]:
pick = site_data()

In [ ]:
pick.get_h1s(i, {'vp3', 'ant', 'aod'}, del_ext={'cnt.h1', '_32.cnt'}, trg_dir=trg))

In [ ]:
#erp_peak_mover()
def peak_pick_move(self, source_path, exp_to_move, 
                   dict_of_lists, good_dir, rej_dir):
    '''creates a dictionary of ids & extensions associated with that id.
       eventually moves these files to directory created in concat_peaks_path.
       requires user input.'''

    self.source_path = source_path
    self.exp_to_move = exp_to_move
    self.dict_of_lists = dict_of_lists
    self.good_dir = good_dir
    self.rej_dir = rej_dir

    yes = ['yes', 'y', 'YES', 'Y']
    no = ['no', 'n', 'NO', 'N']  
    concat_exp_path = os.path.join(source_path, exp_to_move)

    count_remove = 0
    count_keep = 0
    print(exp_to_move, 'FILES TO BE MOVED', '\n')           
    for k,v in dict_of_lists.items():
        if 'avg.mt' not in v:
            count_remove +=1
            rej = '*' + k + '*'
            print('\n', k,v)
            wildcard_path_rej = os.path.join(concat_exp_path, rej)
            for name in glob(wildcard_path_rej): #/active_projects/anthony/peaks_dir/ant/*40360200*
                answer=input('Move to REJECTS folder?\n\n')
                if answer in yes:
                    shutil.copy(os.path.join(wildcard_path_rej, name), rej_dir)
                if answer in no:
                    pass
        if 'avg.mt' in v:
            count_keep +=1
            keep = '*' + k + '*'
            print('\n', k,v)
            wildcard_path = os.path.join(concat_exp_path, keep)
            for name in glob(wildcard_path): #/active_projects/anthony/peaks_dir/ant/*40360200* 
                answer=input('Move to NON-REJECTS folder?\n\n')
                if answer in yes:
                    shutil.copy(os.path.join(wildcard_path, name), good_dir)
                if answer in no:
                    pass

    print('\n', count_keep, 'subs accepted',
     '\n', count_remove, 'subs rejected', '\n\n\n')